<a href="https://colab.research.google.com/github/yaircharit/OU-MS-of-CS/blob/CompVis-FinalProject/Font_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Progress

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


To Orgenize the database I seperated each char in every photo (only in the first image for starter.

In [ ]:
import numpy as np
import h5py
db = h5py.File('/content/drive/MyDrive/SynthText.h5')
validation_number = 0.1*len(db['data'])

# train =db['data'][:validation_number]
im_names = list(db['data'].keys())
im = im_names[0]
print(im)
img = db['data'][im][:]
font = db['data'][im].attrs['font']
txt = db['data'][im].attrs['txt']
charBB = db['data'][im].attrs['charBB'].transpose().astype(int)
wordBB = db['data'][im].attrs['wordBB']

chars = []
for i in range(len(charBB)):
  chars.append({'loc': charBB[i], 'font': font[i]})
  
# print(chars)

# for attr in db['data'][im].attrs:
#   print(attr)
#   print(db['data'][im].attrs[attr])

ant+hill_102.jpg_0


Getting the image of a letter

In [ ]:
import cv2 as cv
from google.colab.patches import cv2_imshow

cv2_imshow(img)
coords = chars[15]['loc'].transpose()
x_range = (int(min(coords[0])),int(max(coords[0])))
y_range = (int(min(coords[1])),int(max(coords[1])))
letter_img = img[y_range[0]:y_range[1],x_range[0]:x_range[1]]

res = cv.cvtColor(letter_img, cv.COLOR_RGB2GRAY)
# res = cv.GaussianBlur(res, (3, 3), 0)
# res = cv.bitwise_not(res)
print(res)
temp = []
s= sum([sum(row)/len(row) for row in res])/len(res)*1.05
for row in res:
  row[row <s]=0
print(chars[15]['font'])
cv2_imshow(res)

Now to sum everything so far:
We reorganized the dataset to so that for each image there'll be a list of all letters and its information (location, font, image)

To really start processing the entire DB these tasks needs to be in functions, So i polished their functionalities and improved uppon them while isolating them to stand-alone functions

# Imports

In [203]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
from math import ceil
from google.colab import drive
drive.mount('/content/drive/')

# Functions

In [207]:
def subimage(image, center, wNh, theta):
   ''' 
   Rotates OpenCV image around center with angle theta (in deg)
   then crops the image according to width and height.
   '''

   # Uncomment for theta in radians
   #theta *= 180/np.pi

   width,height = (round(wNh[0]),round(wNh[1]))
   shape = ( image.shape[1], image.shape[0] ) # cv2.warpAffine expects shape in (length, height)

   matrix = cv.getRotationMatrix2D( center=center, angle=theta, scale=1 )
   image = cv.warpAffine( src=image, M=matrix, dsize=shape )
   
   x = int( center[0] - width/2  )
   y = int( center[1] - height/2 )
   
   image = image[ y:y+height, x:x+width ]
   return image

In [192]:
def get_letter_image(img,char_coords):
  """
  Gets the specified letter from image and straightens the image
  :param img:   The image to crop letter from
  :char_coords: Letter's BoundBox coordinations
  :return:      A good image of the subimage in char_coords
  """
  rect = cv.minAreaRect(char_coords)
  img_crop = subimage(img,*rect)
  img_crop = cv.rotate(img_crop, cv.cv2.ROTATE_90_COUNTERCLOCKWISE)

  return img_crop

In [209]:
def get_letters_info(img_data):
  """
  gets an image and its data and returns an organized dataset for each letter in the image
  :param img_data:  db['data'][im], contains 'font' and 'charBB'
  :return:          dataset of letters. letter={'font':'...','loc':[coords], 'img': [image]}
  :rtype:           list
  """
  img = img_data[:]
  font = img_data.attrs['font']
  charBB = img_data.attrs['charBB'].transpose().astype(int)

  chars = []
  for i in range(len(charBB)):
    letter_img = get_letter_image(img,charBB[i])
    chars.append({'loc': charBB[i], 'font': font[i], 'img': letter_img})
  return chars
  

# Initialize Database and setup data structures

In [211]:

db = h5py.File('/content/drive/MyDrive/SynthText.h5')
validation_number = 0.1*len(db['data'])
im_names = list(db['data'].keys())

imgs = []
all_chars = []
for img_name in im_names:
  chars = get_letters_info(db['data'][img_name])
  all_chars += chars
  imgs.append({'img': db['data'][img_name],'chars':chars})
print(len(all_chars))

28197


## Test dataset

In [210]:
cv2_imshow(all_chars[15]['img'])
# print(all_chars[0])